In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import required libraries**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
#import missingno as msno


%matplotlib inline
#plt.style.use('seaborn-darkgrid')
#palette = plt.get_cmap('Set2')

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

**Getting the Data**

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
#Creating a copy of the train and test datasets
c_test  = test.copy()
c_train  = train.copy()

**Getting info of data**

In [ ]:
train.head(10)

In [ ]:
test.head(10)

In [ ]:
train.info()

In [ ]:
test.info()

#### for numerical data

In [ ]:
c_train.describe()

#### for categorical data

In [ ]:
c_train.describe(exclude='number')

### Obsevation_1
Some features have low variance  as majority of data is of 1 category.
We will check if these features have significance else we will drop them. 
These features are : ['Street','LandContour','Utilities','LandSlope','Condition2','RoofMatl','ExterCond','BsmtCond','Heating','CentralAir','Electrical','Functional','GarageQual','GarageCond','PavedDrive','Condition1','SaleType']

* Concat Train and Test datasets

In [ ]:
c_train['train']  = 1
c_test['train']  = 0
df = pd.concat([c_train, c_test], axis=0,sort=False)        

# Data preprocessing

#### Correlation matrix (heatmap style)

In [ ]:
#correlation matrix
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
SalePrice = pd.DataFrame(corrmat['SalePrice'].sort_values(ascending=False))
SalePrice

In [ ]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

* there are some highly correlated feature pairs ,we keep one from each pair 
* #### Saleprice is highly correlated with OverallQual
* #### GarageArea logically has a great relationship with GarageCars,choose only one,'GarageCars'
* #### Have the smallest connection YearBuilt and TotRmsAbvGrd
* #### Also highly correlated 1stFirSF and TotalBsmtSF,as we choose TotalBsmtSF 
* #### TotRmsAbvGrd is highly correlated with GrLivArea

In [ ]:
df = df.drop(['GarageArea','TotRmsAbvGrd' , '1stFlrSF' ],axis=1)

### Scatter plots between 'SalePrice' and correlated variables 

In [ ]:
#scatterplot
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols], height = 2.5)
plt.show();

### Let drop  features we discussed in Observation1 because  they have low variance and are poorly coorelated to salesprice hence provide no valuable info.

In [ ]:
df = df.drop(['Street','LandContour','Utilities','LandSlope','Condition2','RoofMatl','ExterCond','BsmtCond','Heating','CentralAir','Electrical','Functional','GarageQual','GarageCond','PavedDrive','Condition1','SaleType'],axis=1)

**Let's find and fill in the missing data**

In [ ]:
#missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(25)

**We can safely remove these features as they are not important and do not have a high correlation.**

In [ ]:
df =df.drop((missing_data[missing_data['Total'] > 5]).index, axis=1)
print(df.isnull().sum().max())

In [ ]:
df.isnull().sum()

In [ ]:
# numeric data
numeric_missed = ['BsmtFinSF1',
                  'BsmtFinSF2',
                  'BsmtUnfSF',
                  'TotalBsmtSF',
                  'BsmtFullBath',
                  'BsmtHalfBath',
                  'GarageCars']

for feature in numeric_missed:
    df[feature].fillna(0, inplace=True)

In [ ]:
# categorical data
categorical_missed = ['Exterior1st',
                  'Exterior2nd',
                  'MSZoning',
                  'KitchenQual']

for feature in categorical_missed:
    df[feature].fillna(df[feature].mode()[0], inplace=True)

**Check for any  null values **

In [ ]:
df.isnull().sum().max()

**There are no more null values**
**lets check normality of 'SalesPrice'**



In [ ]:
plt.figure(figsize=(12, 7))

sns.displot(train['SalePrice']).set(ylabel=None, xlabel=None)
plt.title('House price distribution histogram', fontsize=18)
plt.show()

 'SalePrice' is not normal. It shows 'peakedness', positive skewness
 fix it with  log transformation

In [ ]:
#train['SalePrice'] = np.log1p(train['SalePrice'])

#### Fix The Skewness in the other features

In [ ]:
from scipy.stats import skew

In [ ]:

#numeric = df.dtypes[df.dtypes != 'object'].index
#skewed = df[numeric].apply(lambda col: skew(col)).sort_values(ascending=False)
#skewed = skewed[abs(skewed) > 0.5]

#for feature in skewed.index:
    #df[feature] = np.log1p(df[feature])

#### At last  dummy variables

In [ ]:
df = pd.get_dummies(df)
df

In [ ]:
df = df.drop(['Id',],axis=1)

In [ ]:
#Scaler = StandardScaler()
#all_scaled = pd.DataFrame(Scaler.fit_transform(df))

df_train = pd.DataFrame(df[:1460])
df_test = pd.DataFrame(df[1460:2920])

In [ ]:
#df_final = df.drop(['Id',],axis=1)

#df_train = df_final[df_final['train'] == 1]
#df_train = df_train.drop(['train',],axis=1)


#df_test = df[~df_train]
#df_test = df_test.drop(['Id'],axis=1)
#df_test = df_test.drop(['train',],axis=1)

**train-test data**

In [ ]:
target= train['SalePrice']

## Modeling

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_train,target,test_size=0.33,random_state=0)

In [ ]:
xgb =XGBRegressor(learning_rate=0.1,n_estimators=100,reg_alpha=0.001,reg_lambda=0.000001,n_jobs=-1,min_child_weight=3,subsample=0.9,max_depth=5,colsample_bytree=0.2)

In [ ]:
rf = RandomForestRegressor()

In [ ]:
xgb.fit(X_train,y_train) 
rf.fit(X_train,y_train)

In [ ]:
df_tn = pd.DataFrame()
df_tn['xgb_pred'] = xgb.predict(X_test)
df_tn['rf_pred'] = rf.predict(X_test)

In [ ]:
df_ts = pd.DataFrame()
df_ts['xgb_pred'] = xgb.predict(df_test)
df_ts['rf_pred'] = rf.predict(df_test)

In [ ]:
from sklearn.linear_model import LinearRegression

# Create linear regression model without the intercept
lr = LinearRegression(fit_intercept=False)

In [ ]:
lr.fit(df_tn[['xgb_pred', 'rf_pred']], y_test)
df_ts['stacking'] = lr.predict(df_ts[['xgb_pred', 'rf_pred']])

In [ ]:
#df_f['blend'] = (df_f['xgb_pred'] + df_f['rf_pred']) / 2
#print(df_f[['xgb_pred', 'rf_pred', 'blend']].head(3))

In [ ]:
print(lr.coef_)

**Submission**

In [ ]:
submission = pd.DataFrame({
        "Id": test["Id"],
        "SalePrice": df_ts['stacking']
    })
submission.to_csv('submission.csv', index=False)